In [ ]:
import os
import numpy as np
from scipy import special
from numpy.random import randn
from numpy.matlib import repmat
from scipy.linalg import orth
import pandas as pd
! pip install ppca-rs
! pip install maturin
from ppca_rs import Dataset, PPCATrainer
from numpy.linalg import inv
from numpy import transpose as tr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 8.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 63.3 MB/s eta 0:00:00


# PPCA Class and Sources


[first PPCA class](https://github.com/ymcdull/ppca/blob/master/ppca.py)

In [ ]:
## updating W, X and tau when doing inference
class PPCA1:
    ## Y: input continuous data with shape (N, M)
    ## D: number of ppca components
    def __init__(self, D = 2, n_iters = 100, verbose = False):
        self.D = D
        self.n_iters = n_iters
        self.verbose = verbose

    def _init_paras(self, N, M, D):
        self.a = 1.0
        self.b = 1.0
        self.e_tau = self.a / self.b
        self.e_w = randn(M, D)
        self.e_wwt = np.zeros((D, D, M))
        for m in range(M):
            ## use np.newaxis here to transfer numpy array from 1D to 2D
            self.e_wwt[:, :, m] = np.eye(D) + self.e_w[m, :][np.newaxis].T.dot(self.e_w[m, :][np.newaxis])

        self.tol = 1e-3
        self.lbs = []
        self.e_X = np.zeros((N, D))
        self.e_XXt = np.zeros((D, D, N))

    def _update_X(self, Y, N, D):
        self.sigx = np.linalg.inv(np.eye(D) + self.e_tau * np.sum(self.e_wwt, axis = 2))
        for n in range(N):
            print(np.tile(Y.iloc[:, n], (D, 1)).T.shape, self.e_w.shape)
            self.e_X[n, :] = self.e_tau * self.sigx.dot(np.sum(self.e_w * np.tile(Y.iloc[:, n], (D, 1)).T, axis = 0))
            self.e_XXt[:, :, n] = self.sigx + self.e_X[n, :][np.newaxis].T.dot(self.e_X[n, :][np.newaxis])

    def _update_W(self, Y, M, D):
        self.sigw = np.linalg.inv(np.eye(D) + self.e_tau * np.sum(self.e_XXt, axis = 2))
        for m in range(M):
            self.e_w[m, :] = self.e_tau * self.sigw.dot(np.sum(self.e_X * np.tile(Y[:, m], (D, 1)).T, axis = 0))
            self.e_wwt[:, :, m] = self.sigw + self.e_w[m, :][np.newaxis].T.dot(self.e_w[m, :][np.newaxis])

    def _update_tau(self, Y, M, N):
        self.e = self.a + N * M * 1.0 / 2
        outer_expect = 0
        for n in range(N):
            for m in range(M):
                outer_expect = outer_expect \
                                + np.trace(self.e_wwt[:, :, m].dot(self.sigx)) \
                                + self.e_X[n, :][np.newaxis].dot(self.e_wwt[:, :, m]).dot(self.e_X[n, :][np.newaxis].T)[0][0]
        self.f = self.b + 0.5 * np.sum(Y ** 2) - np.sum(Y * self.e_w.dot(self.e_X.T).T) + 0.5 * outer_expect
        self.e_tau = self.e / self.f
        self.e_log_tau = np.mean(np.log(np.random.gamma(self.e, 1/self.f, size=1000)))

    def lower_bound(self,Y, M, N, D):
        LB = self.a * np.log(self.b) + (self.a - 1) * self.e_log_tau - self.b * self.e_tau - special.gammaln(self.a)
        LB = LB - (self.e * np.log(self.f) + (self.e - 1) * self.e_log_tau - self.f * self.e_tau - special.gammaln(self.e))
        for n in range(N):
            LB = LB + (-(D*0.5)*np.log(2*np.pi) - 0.5 * (np.trace(self.sigx) + self.e_X[n, :][np.newaxis].dot(self.e_X[n, :][np.newaxis].T)[0][0]))
            LB = LB - (-(D*0.5)*np.log(2*np.pi) - 0.5 * np.log(np.linalg.det(self.sigx)) - 0.5 * D)
        for m in range(M):
            LB = LB + (-(D*0.5)*np.log(2*np.pi) - 0.5 * (np.trace(self.sigw) + self.e_w[m, :][np.newaxis].dot(self.e_w[m, :][np.newaxis].T)[0][0]))
            LB = LB - (-(D*0.5)*np.log(2*np.pi) - 0.5 * np.log(np.linalg.det(self.sigw)) - 0.5 * D)
        outer_expect = 0
        for n in range(N):
            for m in range(M):
                outer_expect = outer_expect \
                                + np.trace(self.e_wwt[:, :, m].dot(self.sigx)) \
                                + self.e_X[n, :][np.newaxis].dot(self.e_wwt[:, :, m]).dot(self.e_X[n, :][np.newaxis].T)[0][0]

        LB = LB + ( \
            -(N * M * 1.0 / 2) * np.log(2 * np.pi) + (N * M * 1.0 / 2) * self.e_log_tau \
            - 0.5 * self.e_tau * (np.sum(Y**2) - 2 * np.sum(Y * self.e_w.dot(self.e_X.T).T) + outer_expect))
        return LB

    def _update(self, Y, N, M, D):
        self._update_X(Y, N, D)
        self._update_W(Y, M, D)
        self._update_tau(Y, M, N)
        LB = self.lower_bound(Y, M, N, D)
        self.lbs.append(LB)
        if self.verbose:
            print("Lower bound: {}".format(LB))


    def fit(self, Y):
        N, M = Y.shape
        D = self.D
#         temporarily comment these two lines out
#         if not D:
#             D = N
        self._init_paras(N, M, D)

        for it in range(self.n_iters):
            self._update(Y, N, M, D)
            if self.verbose:
                print(it)
            if it >= 1:
                if np.abs(self.lbs[it] - self.lbs[it - 1]) < self.tol:
                    break

    def recover(self):
        return self.e_X.dot(self.e_w.T)

[second PPCA class](https://github.com/allentran/pca-magic/)
also the implementation here [Market Data Analysis applying PPCA in Kaggle](https://www.kaggle.com/code/dandm1/market-data-analysis-applying-ppca/notebook)

In [ ]:
class PPCA2():

    def __init__(self):

        self.raw = None
        self.data = None
        self.C = None
        self.means = None
        self.stds = None
        self.eig_vals = None

    def _standardize(self, X):

        if self.means is None or self.stds is None:
            raise RuntimeError("Fit model first")

        return (X - self.means) / self.stds

    def fit(self, data, d=None, tol=1e-4, min_obs=10, verbose=False):

        self.raw = data
        self.raw[np.isinf(self.raw)] = np.max(self.raw[np.isfinite(self.raw)])

        valid_series = np.sum(~np.isnan(self.raw), axis=0) >= min_obs

        data = self.raw[:, valid_series].copy()
        N = data.shape[0]
        D = data.shape[1]

        self.means = np.nanmean(data, axis=0)
        self.stds = np.nanstd(data, axis=0)

        data = self._standardize(data)
        observed = ~np.isnan(data)
        missing = np.sum(~observed)
        data[~observed] = 0

        # initial

        if d is None:
            d = data.shape[1]

        if self.C is None:
            C = np.random.randn(D, d)
        else:
            C = self.C
        CC = np.dot(C.T, C)
        X = np.dot(np.dot(data, C), np.linalg.inv(CC))
        recon = np.dot(X, C.T)
        recon[~observed] = 0
        ss = np.sum((recon - data)**2)/(N*D - missing)

        v0 = np.inf
        counter = 0

        while True:

            Sx = np.linalg.inv(np.eye(d) + CC/ss)

            # e-step
            ss0 = ss
            if missing > 0:
                proj = np.dot(X, C.T)
                data[~observed] = proj[~observed]
            X = np.dot(np.dot(data, C), Sx) / ss

            # m-step
            XX = np.dot(X.T, X)
            C = np.dot(np.dot(data.T, X), np.linalg.pinv(XX + N*Sx))
            CC = np.dot(C.T, C)
            recon = np.dot(X, C.T)
            recon[~observed] = 0
            ss = (np.sum((recon-data)**2) + N*np.sum(CC*Sx) + missing*ss0)/(N*D)

            # calc diff for convergence
            det = np.log(np.linalg.det(Sx))
            if np.isinf(det):
                det = abs(np.linalg.slogdet(Sx)[1])
            v1 = N*(D*np.log(ss) + np.trace(Sx) - det) \
                + np.trace(XX) - missing*np.log(ss0)
            diff = abs(v1/v0 - 1)
            if verbose:
                print('\rAt iteration {} the diff is {:8.6f} (target {})'.format(counter,diff,tol),end='')
            if (diff < tol) and (counter > 5):
                break

            counter += 1
            v0 = v1


        C = orth(C)
        vals, vecs = np.linalg.eig(np.cov(np.dot(data, C).T))
        order = np.flipud(np.argsort(vals))
        vecs = vecs[:, order]
        vals = vals[order]

        C = np.dot(C, vecs)

        # attach objects to class
        self.C = C
        self.data = data
        self.eig_vals = vals
        self._calc_var()

    def transform(self, data=None):

        if self.C is None:
            raise RuntimeError('Fit the data model first.')
        if data is None:
            return np.dot(self.data, self.C)
        missing = np.isnan(data)
        #Obtain mean of columns as you need, nanmean is just convenient.
        it = 0
        if np.sum(missing) > 0:
            col_mean = np.nanmean(data, axis=0)
            data[missing] = np.take(col_mean, np.where(missing)[0])
            change = 1
            while(change>1e-3):
                CC = np.dot(self.C.T, self.C)
                X = np.dot(np.dot(data, self.C), np.linalg.inv(CC))
                proj = np.dot(X, self.C.T)
                change = np.max(np.abs(data[missing]-proj[missing]))
                print('\rIteration {}. Change is {:6.4f}'.format(it,change),end='')
                data[missing] = proj[missing]
                it += 1
        return np.dot(data, self.C)

    def _calc_var(self):

        if self.data is None:
            raise RuntimeError('Fit the data model first.')

        data = self.data.T

        # variance calc
        var = np.nanvar(data, axis=1)
        total_var = var.sum()
        self.var_exp = self.eig_vals.cumsum() / total_var

    def save(self, fpath):

        np.save(fpath, self.C)

    def load(self, fpath):

        assert os.path.isfile(fpath)

        self.C = np.load(fpath)

PPCA3 from this source [CangerMuellerPPCA-Github](https://github.com/cangermueller/ppca/blob/master/src/pca/ppca.py)

In [ ]:
class PPCA3(object):
    def __init__(self, q=2, sigma=1.0):
        self.q = q
        self.prior_sigma = sigma

    def fit(self, y, em=False):
        self.y = y
        self.p = y.shape[0]
        self.n = y.shape[1]
        if em:
            [self.w, self.mu, self.sigma] = self.__fit_em()
        else:
            [self.w, self.mu, self.sigma] = self.__fit_ml()

    def transform(self, y=None):
        if y is None:
            y = self.y
        [w, mu, sigma] = [self.w, self.mu, self.sigma]
        m = tr(w).dot(w) + sigma * np.eye(w.shape[1])
        m = inv(m)
        x = m.dot(tr(w)).dot(y - mu)
        return x

    def fit_transform(self, *args, **kwargs):
        self.fit(*args, **kwargs)
        return self.transform()

    def transform_infers(self, x=None, noise=False):
        if x is None:
            x = self.transform()
        [w, mu, sigma] = [self.w, self.mu, self.sigma]
        y = w.dot(x) + mu
        if noise:
            for i in xrange(y.shape[1]):
                e = np.random.normal(0, sigma, y.shape[0])
                y[:, i] += e
        return y

    def __ell(self, w, mu, sigma, norm=True):
        m = inv(tr(w).dot(w) + sigma * np.eye(w.shape[1]))
        mw = m.dot(tr(w))
        ll = 0.0
        for i in xrange(self.n):
            yi = self.y[:, i][:, np.newaxis]
            yyi = yi - mu
            xi = mw.dot(yyi)
            xxi = sigma * m + xi.dot(tr(xi))
            ll += 0.5 * np.trace(xxi)
            if sigma > 1e-5:
                ll += (2 * sigma)**-1 * float(tr(yyi).dot(yyi))
                ll -= sigma**-1 * float(tr(xi).dot(tr(w)).dot(yyi))
                ll += (2 * sigma)**-1 * np.trace(tr(w).dot(w).dot(xxi))
        if sigma > 1e-5:
            ll += 0.5 * self.n * self.p * np.log(sigma)
        ll *= -1.0
        if norm:
            ll /= float(self.n)
        return ll

    def __fit_em(self, maxit=20):
        w = np.random.rand(self.p, self.q)
        mu = np.mean(self.y, 1)[:, np.newaxis]
        sigma = self.prior_sigma
        ll = self.__ell(w, mu, sigma)

        yy = self.y - mu
        s = self.n**-1 * yy.dot(tr(yy))
        for i in range(maxit):
            m = inv(tr(w).dot(w) + sigma * np.eye(self.q))
            t = inv(sigma * np.eye(self.q) + m.dot(tr(w)).dot(s).dot(w))
            w_new = s.dot(w).dot(t)
            sigma_new = self.p**-1 * np.trace(s - s.dot(w).dot(m).dot(tr(w_new)))
            ll_new = self.__ell(w_new, mu, sigma_new)
            print("{:3d}  {:.3f}".format(i + 1, ll_new))
            w = w_new
            sigma = sigma_new
            ll = ll_new
        return (w, mu, sigma)

    def __fit_ml(self):
        mu = np.mean(self.y, 1)[:, np.newaxis]
        [u, s, v] = np.linalg.svd(self.y - mu)
        if self.q > len(s):
            ss = np.zeros(self.q)
            ss[:len(s)] = s
        else:
            ss = s[:self.q]
        ss = np.sqrt(np.maximum(0, ss**2 - self.prior_sigma))
        w = u[:, :self.q].dot(np.diag(ss))
        if self.q < self.p:
            sigma = 1.0 / (self.p - self.q) * np.sum(s[self.q:]**2)
        else:
            sigma = 0.0
        return (w, mu, sigma)

Other PPCA sources [ppca-rs](https://pypi.org/project/ppca-rs/)

# Data Import, Exploration, and Pre-processing

In [ ]:
! unzip kidney_disease_pola.zip
# ! unzip dataset.csv.zip

Archive:  kidney_disease_pola.zip
  inflating: kidney_disease.csv      


In [ ]:
df = pd.read_csv(f'kidney_disease.csv',sep=",")
# df_hosp = pd.read_csv(f'dataset.csv',sep=",")

In [ ]:
tmp_df =  pd.DataFrame()

for idx,i in enumerate(df) :
  if pd.api.types.is_string_dtype(df[i]):
    df[i] = df[i].str.replace("\t", "")
    df[i] = df[i].str.replace(" ", "")
    df[i] = df[i].replace(np.nan, np.nan)
    df[i] = df[i].replace('?', np.nan)
    if i == "wc" or i == "rc" or i == "pcv":
      df[i] = df[i].astype(float)

df['classification'] = df['classification'].map({'notckd': 0, 'ckd': 1})

for idx,i in enumerate(df) :
  if pd.api.types.is_string_dtype(df[i]):
    tmp_df[i] = df[i]
    print(idx, i, len(df[i].unique()), "\n", df[i].unique())

    df = df.drop([i], axis=1)

tmp_df_dummies1 = pd.get_dummies(tmp_df)
print(tmp_df_dummies1.shape)
print(tmp_df_dummies1)

6 rbc 3 
 [nan 'normal' 'abnormal']
7 pc 3 
 ['normal' 'abnormal' nan]
8 pcc 3 
 ['notpresent' 'present' nan]
9 ba 3 
 ['notpresent' 'present' nan]
19 htn 3 
 ['yes' 'no' nan]
20 dm 3 
 ['yes' 'no' nan]
21 cad 3 
 ['no' 'yes' nan]
22 appet 3 
 ['good' 'poor' nan]
23 pe 3 
 ['no' 'yes' nan]
24 ane 3 
 ['no' 'yes' nan]
(400, 20)
     rbc_abnormal  rbc_normal  pc_abnormal  pc_normal  pcc_notpresent  \
0               0           0            0          1               1   
1               0           0            0          1               1   
2               0           1            0          1               1   
3               0           1            1          0               0   
4               0           1            0          1               1   
..            ...         ...          ...        ...             ...   
395             0           1            0          1               1   
396             0           1            0          1               1   
397           

In [ ]:
tmp_df =  pd.DataFrame()
print(df_hosp.shape)
for idx,i in enumerate(df_hosp) :
  if pd.api.types.is_string_dtype(df_hosp[i]):
  #   df_hosp[i] = df_hosp[i].str.replace("\t", "")
  #   df_hosp[i] = df_hosp[i].str.replace(" ", "")
  #   df_hosp[i] = df_hosp[i].replace(np.nan, np.nan)
  #   df_hosp[i] = df_hosp[i].replace('?', np.nan)
  #   if i == "wc" or i == "rc" or i == "pcv":
  #     df[i] = df[i].astype(float)
    tmp_df[i] = df_hosp[i]
    print(idx, i, len(df_hosp[i].unique()), "\n", df_hosp[i].unique())

    df_hosp = df_hosp.drop([i], axis=1)
print(tmp_df)
tmp_df_dummies = pd.get_dummies(tmp_df)
print(tmp_df_dummies.shape)

(91713, 85)
6 ethnicity 7 
 ['Caucasian' nan 'Hispanic' 'African American' 'Asian' 'Native American'
 'Other/Unknown']
7 gender 3 
 ['M' 'F' nan]
9 icu_admit_source 6 
 ['Floor' 'Accident & Emergency' 'Operating Room / Recovery'
 'Other Hospital' 'Other ICU' nan]
11 icu_stay_type 3 
 ['admit' 'readmit' 'transfer']
12 icu_type 8 
 ['CTICU' 'Med-Surg ICU' 'CCU-CTICU' 'Neuro ICU' 'MICU' 'SICU'
 'Cardiac ICU' 'CSICU']
81 apache_3j_bodysystem 12 
 ['Sepsis' 'Respiratory' 'Metabolic' 'Cardiovascular' 'Trauma'
 'Neurological' 'Gastrointestinal' 'Genitourinary' nan 'Hematological'
 'Musculoskeletal/Skin' 'Gynecological']
82 apache_2_bodysystem 11 
 ['Cardiovascular' 'Respiratory' 'Metabolic' 'Trauma' 'Neurologic'
 'Gastrointestinal' 'Renal/Genitourinary' nan 'Undefined diagnoses'
 'Haematologic' 'Undefined Diagnoses']
       ethnicity gender           icu_admit_source icu_stay_type  \
0      Caucasian      M                      Floor         admit   
1      Caucasian      F                   

In [ ]:
df_dummies = pd.concat([df.iloc[:,:-1], tmp_df_dummies1, df.iloc[:,-1]], axis=1)
print(df_dummies.describe(include='all'))

               id         age          bp          sg          al          su  \
count  400.000000  391.000000  388.000000  353.000000  354.000000  351.000000   
mean   199.500000   51.483376   76.469072    1.017408    1.016949    0.450142   
std    115.614301   17.169714   13.683637    0.005717    1.352679    1.099191   
min      0.000000    2.000000   50.000000    1.005000    0.000000    0.000000   
25%     99.750000   42.000000   70.000000    1.010000    0.000000    0.000000   
50%    199.500000   55.000000   80.000000    1.020000    0.000000    0.000000   
75%    299.250000   64.500000   80.000000    1.020000    2.000000    0.000000   
max    399.000000   90.000000  180.000000    1.025000    5.000000    5.000000   

              bgr          bu          sc         sod  ...     dm_yes  \
count  356.000000  381.000000  383.000000  313.000000  ...  400.00000   
mean   148.036517   57.425722    3.072454  137.528754  ...    0.34250   
std     79.281714   50.503006    5.741126   10.4087

In [ ]:
df_hosp_dummies = pd.concat([df_hosp.iloc[:,:-1], tmp_df_dummies,df_hosp.iloc[:,-1]], axis=1)
print(df_hosp_dummies.describe(include='all'))

        encounter_id     patient_id   hospital_id           age           bmi  \
count   91713.000000   91713.000000  91713.000000  87485.000000  88284.000000   
mean    65606.079280   65537.131464    105.669262     62.309516     29.185818   
std     37795.088538   37811.252183     62.854406     16.775119      8.275142   
min         1.000000       1.000000      2.000000     16.000000     14.844926   
25%     32852.000000   32830.000000     47.000000     52.000000     23.641975   
50%     65665.000000   65413.000000    109.000000     65.000000     27.654655   
75%     98342.000000   98298.000000    161.000000     75.000000     32.930206   
max    131051.000000  131051.000000    204.000000     89.000000     67.814990   

       elective_surgery        height        icu_id  pre_icu_los_days  \
count      91713.000000  90379.000000  91713.000000      91713.000000   
mean           0.183736    169.641588    508.357692          0.835766   
std            0.387271     10.795378    228.989661

# PPCA Implementation for Dataset

In [ ]:
df_noclass = df_dummies.copy()
df_noclass = df_noclass.drop(["classification"], axis=1)
df_noclass = df_noclass.drop(["id"], axis=1)
dataset_noclass_np = df_noclass.to_numpy()
print(dataset_noclass_np.shape)

dataset_noclass_np_classname = [ i for i in df_noclass]
print(dataset_noclass_np_classname)

(400, 34)
['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'rbc_abnormal', 'rbc_normal', 'pc_abnormal', 'pc_normal', 'pcc_notpresent', 'pcc_present', 'ba_notpresent', 'ba_present', 'htn_no', 'htn_yes', 'dm_no', 'dm_yes', 'cad_no', 'cad_yes', 'appet_good', 'appet_poor', 'pe_no', 'pe_yes', 'ane_no', 'ane_yes']


In [ ]:
df_hosp_noclass = df_hosp_dummies.copy()
df_hosp_noclass = df_hosp_noclass.drop(["hospital_death"], axis=1)
df_hosp_noclass = df_hosp_noclass.drop(["encounter_id","patient_id","icu_id",'Unnamed: 83'], axis=1)
dataset_hosp_noclass_np = df_hosp_noclass.to_numpy()
print(dataset_hosp_noclass_np.shape)

dataset_hosp_noclass_np_classname = [ i for i in df_hosp_noclass]
print(dataset_hosp_noclass_np_classname)

(91713, 118)
['hospital_id', 'age', 'bmi', 'elective_surgery', 'height', 'pre_icu_los_days', 'weight', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'apache_post_operative', 'arf_apache', 'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache', 'heart_rate_apache', 'intubated_apache', 'map_apache', 'resprate_apache', 'temp_apache', 'ventilated_apache', 'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_max', 'h1_mbp_min', 'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min', 'h1_resprate

In [ ]:
import numpy as np
from ppca_rs import Dataset, PPCATrainer

nan_counter = 0
for idx1,row in enumerate(dataset_noclass_np):
  for idx2,data in enumerate(row) :
    if np.isnan(data) == True:
      nan_counter+=1
print("normal data nan values count is", nan_counter, "\n")

dataset = Dataset(dataset_noclass_np.T)

# Train the model (convenient edition!):
model = PPCATrainer(dataset).train(state_size=8, n_iters=10)

# model = PPCAModel.init(8, dataset)

# And now, here is a free sample of what you can do:

# Extrapolates the missing values with the most probable values:
extrapolated = model.extrapolate(dataset)

# Smooths (removes noise from) samples and fills in missing values:
# extrapolated_smooths = model.filter_extrapolate(dataset)

# ... go back to numpy:
eextrapolated_np = extrapolated.numpy()

nan_counter_ppca = 0
print("exp",extrapolated)
print("np_exp",eextrapolated_np, eextrapolated_np.shape)
for idx1,row in enumerate(eextrapolated_np):
  for idx2,data in enumerate(row) :
    if np.isnan(data) == True:
      print("NaN found at", idx1,idx2)
      nan_counter_ppca += 1
print("\nafter ppca, nan counter on dataset is ", nan_counter_ppca, "\n")

normal data nan values count is 778 

Masked PPCA iteration 1: aic=673387623.9084002
Masked PPCA iteration 2: aic=6682.001524837114
Masked PPCA iteration 3: aic=6305.105929583359
Masked PPCA iteration 4: aic=4392.588227654591
Masked PPCA iteration 5: aic=3595.9938204598748
Masked PPCA iteration 6: aic=3267.6576306824154
Masked PPCA iteration 7: aic=2872.4823082931675
Masked PPCA iteration 8: aic=2527.8059075312603
Masked PPCA iteration 9: aic=2358.716738274924
Masked PPCA iteration 10: aic=2281.439133983507
exp <ppca_rs.Dataset object at 0x7f44cd2ad8b0>
np_exp [[48.     7.    62.    ... 12.    17.    58.   ]
 [80.    50.    80.    ... 80.    60.    80.   ]
 [ 1.02   1.02   1.01  ...  1.02   1.025  1.025]
 ...
 [ 0.     0.     0.    ...  0.     0.     0.   ]
 [ 1.     1.     0.    ...  1.     1.     1.   ]
 [ 0.     0.     1.    ...  0.     0.     0.   ]] (34, 400)

after ppca, nan counter on dataset is  0 



In [ ]:
import numpy as np
from ppca_rs import Dataset, PPCATrainer

nan_counter2 = 0
for idx1,row in enumerate(dataset_hosp_noclass_np):
  for idx2,data in enumerate(row) :
    if np.isnan(data) == True:
      nan_counter2 +=1
print("normal data nan values count is", nan_counter2, "\n")

dataset2 = Dataset(dataset_hosp_noclass_np.T)

# Train the model (convenient edition!):
model2 = PPCATrainer(dataset2).train(state_size=20, n_iters=10)

# model = PPCAModel.init(8, dataset)

# And now, here is a free sample of what you can do:

# Extrapolates the missing values with the most probable values:
extrapolated2 = model2.extrapolate(dataset2)

# Smooths (removes noise from) samples and fills in missing values:
# extrapolated_smooths = model.filter_extrapolate(dataset)

# ... go back to numpy:
eextrapolated_np2 = extrapolated2.numpy()

nan_counter_ppca2 = 0
print("exp",extrapolated2)
print("np_exp",eextrapolated_np2, eextrapolated_np2.shape)
for idx1,row in enumerate(eextrapolated_np2):
  for idx2,data in enumerate(row) :
    if np.isnan(data) == True:
      print("NaN found at", idx1,idx2)
      nan_counter_ppca2 += 1
print("\nafter ppca, nan counter on dataset is ", nan_counter_ppca2, "\n")

normal data nan values count is 191477 

Masked PPCA iteration 1: aic=768440303.5198591
Masked PPCA iteration 2: aic=1014020.428120418
Masked PPCA iteration 3: aic=507050.99147774564
Masked PPCA iteration 4: aic=480263.27288969426
Masked PPCA iteration 5: aic=2218482.4579171487
Masked PPCA iteration 6: aic=1335469.7465737725
Masked PPCA iteration 7: aic=762469.5112341458
Masked PPCA iteration 8: aic=573778.6481693063
Masked PPCA iteration 9: aic=484215.7451194299
Masked PPCA iteration 10: aic=478387.66311879724
exp <ppca_rs.Dataset object at 0x7fde1ff9a930>
np_exp [[118.          81.         118.         ... 195.          66.
  104.        ]
 [ 68.          77.          25.         ...  48.         156.58387471
   82.        ]
 [ 22.73        27.42        31.95       ...  27.23691351  23.29748133
   22.03125   ]
 ...
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.       

In [ ]:
new_df = pd.DataFrame(eextrapolated_np.T, columns = dataset_noclass_np_classname)
new_df.to_csv("./kidney_disease_ppca_proc.csv")
print(new_df)

      age    bp     sg   al   su         bgr    bu   sc         sod       pot  \
0    48.0  80.0  1.020  1.0  0.0  121.000000  36.0  1.2  146.877761  3.836935   
1     7.0  50.0  1.020  4.0  0.0   43.917331  18.0  0.8  104.493783  1.068779   
2    62.0  80.0  1.010  2.0  3.0  423.000000  53.0  1.8  124.491626  5.987803   
3    48.0  70.0  1.005  4.0  0.0  117.000000  56.0  3.8  111.000000  2.500000   
4    51.0  80.0  1.010  2.0  0.0  106.000000  26.0  1.4  126.546958  2.959277   
..    ...   ...    ...  ...  ...         ...   ...  ...         ...       ...   
395  55.0  80.0  1.020  0.0  0.0  140.000000  49.0  0.5  150.000000  4.900000   
396  42.0  70.0  1.025  0.0  0.0   75.000000  31.0  1.2  141.000000  3.500000   
397  12.0  80.0  1.020  0.0  0.0  100.000000  26.0  0.6  137.000000  4.400000   
398  17.0  60.0  1.025  0.0  0.0  114.000000  50.0  1.0  135.000000  4.900000   
399  58.0  80.0  1.025  0.0  0.0  131.000000  18.0  1.1  141.000000  3.500000   

     ...  dm_no  dm_yes  ca

In [ ]:
new_df_hosp = pd.DataFrame(eextrapolated_np2.T, columns = dataset_hosp_noclass_np_classname)
new_df_hosp.to_csv("./hospital_survivability_ppca_proc.csv")
print(new_df_hosp)

       hospital_id         age        bmi  elective_surgery  height  \
0            118.0   68.000000  22.730000               0.0   180.3   
1             81.0   77.000000  27.420000               0.0   160.0   
2            118.0   25.000000  31.950000               0.0   172.7   
3            118.0   81.000000  22.640000               1.0   165.1   
4             33.0   19.000000  63.598387               0.0   188.0   
...            ...         ...        ...               ...     ...   
91708         30.0   75.000000  23.060250               0.0   177.8   
91709        121.0   56.000000  47.179671               0.0   183.0   
91710        195.0   48.000000  27.236914               0.0   170.2   
91711         66.0  156.583875  23.297481               0.0   154.9   
91712        104.0   82.000000  22.031250               1.0   160.0   

       pre_icu_los_days      weight  apache_2_diagnosis  apache_3j_diagnosis  \
0              0.541667   73.900000               113.0            

In [ ]:
import matplotlib.pyplot as plt

print(model.singular_values.shape, model.singular_values)
print(model.transform.shape, model.transform)
newww_df = pd.DataFrame(model.transform)

print([clss for clss in df.iloc[:,-1]])

colors = {0:'red', 1:'green'}

for i in range(len(model.singular_values)):
  for j in range(i,len(model.singular_values)):
    class_color = [int(clss) for clss in df.iloc[:,-1]]
    fig, ax = plt.subplots()
    ax.scatter(newww_df.iloc[:,i], newww_df.iloc[:,j], c=(class_color.map(colors)))

plt.show()

In [ ]:
import matplotlib.pyplot as plt


print(model2.singular_values.shape, model2.singular_values)
print(model2.transform.shape, model2.transform)
newww_df2 = pd.DataFrame(model2.transform)

# colors = {0:'red', 1:'green'}

# for i in range(len(model2.singular_values)):
#   for j in range(i,len(model2.singular_values)):
#     fig, ax = plt.subplots()
#     ax.scatter(newww_df2.iloc[:,i], newww_df2.iloc[:,j], c=df_hosp['hospital_death'].map(colors))

# plt.show()

(20,) [77.42547718 55.12748001 49.47932797 35.37678418 26.58068054 23.36032739
 19.47103664 17.45172934 15.50738726 13.58684371 13.32657195 12.184127
 10.41927529  8.93023367  8.4263865   7.53026948  6.93624595  4.54968669
  4.377864    2.94492012]
(91713, 20) [[ 1.51325801e+01  2.12603679e+00  2.62705961e+00 ...  4.60129680e-02
   1.74651909e-03 -2.87639004e-02]
 [ 1.54143765e+01 -1.47650886e+00 -4.94076981e+00 ...  5.73103094e-02
  -3.41433067e-02 -2.21730950e-02]
 [ 1.49159426e+01  1.11716904e+01  1.02189977e-01 ...  3.86805349e-03
   8.38532412e-02 -8.29173971e-03]
 ...
 [ 5.41909866e+01 -3.58035110e+01 -2.95438286e+01 ... -3.88611779e-02
  -4.99973222e-02  4.68211656e-02]
 [ 2.24977266e+01 -5.90915656e+00 -1.75175066e+01 ... -2.19591318e-01
  -7.64539253e-02 -9.76250668e-03]
 [ 3.31023382e+01  1.61943075e-01 -2.28916240e+00 ... -4.90069555e-03
  -3.36528590e-02 -1.15375497e-02]]


In [ ]:
transform = np.random.binomial(1.0, 0.1, size=(200, 200))
print(transform.shape)

from ppca_rs import PPCAModel

print("Generating model")

real_model = PPCAModel(
    transform=np.matrix(transform, dtype="float64"),
    isotropic_noise=0.1,
    mean=np.zeros((200, 1), dtype="float64"),
)

print("Generating synthetic sample")
sample = real_model.sample(100_000, 0.2)

print("Initializing model")
model = PPCAModel.init(16, sample)

print("Starting iterations...")

prevs = -99999999
for it in range(24):
    llk = model.llk(sample) / len(sample)
    print(f"At iteration {it + 1} PPCA llk is {llk}")
    if llk - prevs < 1 :
      print("Done creating iterated model")
      break
    else :
      prevs = llk
    model = model.iterate(sample)

print("Model trained")

In [ ]:
model = model.to_canonical()

print("model result", model.transform.shape, model.mean.shape, model)
print("model singular values", model.singular_values.shape, model.singular_values)

inferred = model.infer(sample)
inferred_proccessed = inferred.smoothed_covariances_diagonal(model).numpy() ** 0.5
print("model inferred",inferred_proccessed.shape, inferred_proccessed)

model result (200, 16) (200,) PPCAModel(isotropic_noise=1, transform=array([[ 0.53640847 -2.99371691  0.24689292 ...  1.03654724  0.63440329
   0.80218504]
 [-0.09045675  1.44997762  0.38306477 ...  0.38703624 -0.24720305
   1.80470671]
 [ 0.2644497   0.43294001  0.04256125 ... -0.47070298 -1.35235589
   1.1747701 ]
 ...
 [ 1.35375386 -1.06539963  0.33683727 ...  0.93058822  0.29821911
   0.68754817]
 [ 0.14467425  0.86898957  1.99396288 ... -1.52349158  1.05011599
  -0.17767376]
 [-1.022808   -0.40894655  2.06894134 ... -0.3192753   1.81712973
   0.62209505]], dtype="float32"), mean=narray([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

# RF classification for the PPCA principal components results

W.O PPCA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

default_fillna_df = df_dummies.fillna(df_dummies.mean())
default_fillna_df = default_fillna_df.drop(["id"], axis=1)
default_fillna_df = default_fillna_df.sample(frac = 1)

X_train, X_test, y_train, y_test = train_test_split(
    default_fillna_df.iloc[:,:-1],
    df_dummies.iloc[:,-1],
    test_size=0.25,
    random_state=123
)

rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=22)
rf.fit(X_train, y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print(default_fillna_df,default_fillna_df.shape)
print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

      age     bp        sg        al        su    bgr     bu   sc         sod  \
346  33.0   60.0  1.017408  1.016949  0.450142  130.0   41.0  0.9  141.000000   
266  55.0   80.0  1.020000  0.000000  0.000000  133.0   17.0  1.2  135.000000   
226  64.0  100.0  1.015000  4.000000  2.000000  163.0   54.0  7.2  140.000000   
127  71.0   60.0  1.015000  4.000000  0.000000  118.0  125.0  5.3  136.000000   
19   62.0   60.0  1.015000  1.000000  0.000000  100.0   31.0  1.6  137.528754   
..    ...    ...       ...       ...       ...    ...    ...  ...         ...   
383  80.0   80.0  1.025000  0.000000  0.000000  119.0   46.0  0.7  141.000000   
8    52.0  100.0  1.015000  3.000000  0.000000  138.0   60.0  1.9  137.528754   
313  55.0   80.0  1.020000  0.000000  0.000000  104.0   28.0  0.9  142.000000   
250  40.0   80.0  1.025000  0.000000  0.000000  140.0   10.0  1.2  135.000000   
345  22.0   60.0  1.025000  0.000000  0.000000   97.0   18.0  1.2  138.000000   

          pot  ...  dm_yes 

DATASET 2 W.O PPCA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

default_fillna_df2 = df_hosp_dummies.fillna(df_hosp_dummies.mean())
default_fillna_df2 = default_fillna_df2.drop(["encounter_id","patient_id","icu_id","Unnamed: 83","hospital_death"], axis=1)
default_fillna_df2 = default_fillna_df2.sample(frac = 1)
print(default_fillna_df2)

X_train, X_test, y_train, y_test = train_test_split(
    default_fillna_df2.iloc[:,:-1],
    df_hosp_dummies.iloc[:,-1],
    test_size=0.25,
    random_state=123
)

rf = RandomForestClassifier(n_estimators=25, oob_score=True, random_state=22)
rf.fit(X_train, y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

       hospital_id   age        bmi  elective_surgery  height  \
7525           149  58.0  38.590575                 1   167.6   
56948          176  76.0  39.906048                 0   170.2   
8041           118  60.0  38.293205                 0   182.9   
75186          166  73.0  17.695312                 1   160.0   
28567          147  73.0  15.959610                 0   172.7   
...            ...   ...        ...               ...     ...   
35237           70  73.0  36.290174                 0   165.0   
90989           66  34.0  27.472944                 0   190.5   
85525          188  79.0  29.880373                 0   162.6   
26163          112  71.0  34.429066                 0   170.0   
39556           62  83.0  35.755956                 1   163.0   

       pre_icu_los_days  weight  apache_2_diagnosis  apache_3j_diagnosis  \
7525           5.996528   108.4               214.0              1404.01   
56948          5.969444   115.6               105.0               2

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Out-of-bag score estimate: 0.912
Mean accuracy score: 0.914


FILL NaN WITH PPCA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    new_df.iloc[:,:],
    df_dummies.iloc[:,-1],
    test_size=0.25,
    random_state=123
)

rf = RandomForestClassifier(n_estimators=25, oob_score=True, random_state=22)
rf.fit(X_train, y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(new_df, new_df.shape)
print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

      age    bp     sg   al   su         bgr    bu   sc         sod       pot  \
0    48.0  80.0  1.020  1.0  0.0  121.000000  36.0  1.2  146.877761  3.836935   
1     7.0  50.0  1.020  4.0  0.0   43.917331  18.0  0.8  104.493783  1.068779   
2    62.0  80.0  1.010  2.0  3.0  423.000000  53.0  1.8  124.491626  5.987803   
3    48.0  70.0  1.005  4.0  0.0  117.000000  56.0  3.8  111.000000  2.500000   
4    51.0  80.0  1.010  2.0  0.0  106.000000  26.0  1.4  126.546958  2.959277   
..    ...   ...    ...  ...  ...         ...   ...  ...         ...       ...   
395  55.0  80.0  1.020  0.0  0.0  140.000000  49.0  0.5  150.000000  4.900000   
396  42.0  70.0  1.025  0.0  0.0   75.000000  31.0  1.2  141.000000  3.500000   
397  12.0  80.0  1.020  0.0  0.0  100.000000  26.0  0.6  137.000000  4.400000   
398  17.0  60.0  1.025  0.0  0.0  114.000000  50.0  1.0  135.000000  4.900000   
399  58.0  80.0  1.025  0.0  0.0  131.000000  18.0  1.1  141.000000  3.500000   

     ...  dm_no  dm_yes  ca

DATASET 2 FILL NaN WITH PPCA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(
    new_df_hosp.iloc[:,:],
    df_hosp.iloc[:,-1],
    test_size=0.1,
    random_state=123
)

rf = RandomForestClassifier(n_estimators=25, oob_score=True, random_state=22)
rf.fit(X_train, y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(new_df_hosp, new_df_hosp.shape)
print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

       hospital_id         age        bmi  elective_surgery  height  \
0            118.0   68.000000  22.730000               0.0   180.3   
1             81.0   77.000000  27.420000               0.0   160.0   
2            118.0   25.000000  31.950000               0.0   172.7   
3            118.0   81.000000  22.640000               1.0   165.1   
4             33.0   19.000000  63.598387               0.0   188.0   
...            ...         ...        ...               ...     ...   
91708         30.0   75.000000  23.060250               0.0   177.8   
91709        121.0   56.000000  47.179671               0.0   183.0   
91710        195.0   48.000000  27.236914               0.0   170.2   
91711         66.0  156.583875  23.297481               0.0   154.9   
91712        104.0   82.000000  22.031250               1.0   160.0   

       pre_icu_los_days      weight  apache_2_diagnosis  apache_3j_diagnosis  \
0              0.541667   73.900000               113.0            

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


WITH PPCA PRINCIPAL COMPONENTS TRANSFORMATION

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    newww_df.iloc[:,:],
    df_dummies.iloc[:,-1],
    test_size=0.25,
    random_state=123
)

rf = RandomForestClassifier(n_estimators=25, oob_score=True, random_state=22)
rf.fit(X_train, y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(newww_df, newww_df.shape)
print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

              0         1         2         3         4         5         6  \
0    293.243808  2.122050 -1.171726  0.036323  0.341322 -0.089178 -0.030582   
1    225.684592  1.537855 -2.334857  0.347003  0.286780 -1.070170  0.101263   
2    281.505034 -0.693724  8.482952 -4.684191 -0.656079 -0.349665  0.081582   
3    251.845472  0.357335  0.420515  0.438960  0.491850  0.181797  0.132527   
4    274.473127  1.800969 -1.251130 -0.175125  1.079645  0.288614 -0.126105   
..          ...       ...       ...       ...       ...       ...       ...   
395  251.755301  2.490390 -0.098173  0.266938 -0.090036  0.073948  0.136847   
396  293.323517  2.184215 -2.565576  0.574658 -0.102590 -0.128535  0.547076   
397  248.122336  2.538997 -1.501330 -0.049108  1.363985 -1.474416  0.273925   
398  270.684584  1.173163 -0.553688  0.401099 -0.562628 -1.275029  0.430541   
399  255.570173  3.110520 -1.141709 -0.611712  0.259613  0.363294  0.582855   

            7  
0    0.000089  
1    0.000318  
2  

DATASET 2 WITH PPCA PRINCIPAL COMPONENTS TRANSFORMATION

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    newww_df2.iloc[:,:],
    df_hosp.iloc[:,-1],
    test_size=0.1,
    random_state=123
)

rf = RandomForestClassifier(n_estimators=25, oob_score=True, random_state=22)
rf.fit(X_train, y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print(newww_df2, newww_df2.shape)
print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

              0          1          2         3          4         5   \
0      15.132580   2.126037   2.627060  1.716514   0.760253  0.315627   
1      15.414377  -1.476509  -4.940770  1.751166   1.950002  2.010352   
2      14.915943  11.171690   0.102190 -0.521360   1.352074  0.091054   
3      27.803434   2.272517   1.548813 -3.281464  -0.181441 -1.006776   
4      20.823268 -10.114693  11.355074  4.606056  -1.065756 -1.689137   
...          ...        ...        ...       ...        ...       ...   
91708  24.575705 -11.356237  -0.216515  1.585409   0.571714  1.675602   
91709  16.389128   0.496776   1.603919  1.483112   0.861629  0.091614   
91710  54.190987 -35.803511 -29.543829 -8.944847  15.450441 -9.937488   
91711  22.497727  -5.909157 -17.517507 -1.990424   0.967346  6.844909   
91712  33.102338   0.161943  -2.289162 -5.902585  -1.980586 -0.238761   

             6         7         8         9         10        11        12  \
0      0.518436 -0.516147  0.067919  0.49405

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
